# Text Mining Project

In [ ]:
# Libraries and Packages install
!pip install contractions
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.5 MB/s eta 0:00:00


In [ ]:
## Libraries and Packages import

# Utils
import pandas as pd
import numpy as np
from random import sample

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

# Text related
import string
from string import punctuation
import nltk
import re
import contractions
import gensim
import gensim.models as models
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
nltk.download('punkt')
nltk.download('stopwords')

# Monitoring loops progress
from tqdm import tqdm
from functools import partial
tqdm_par = partial(tqdm, position=0, leave=True)
tqdm.pandas()

# Mute warnings
import warnings
warnings.filterwarnings('ignore')

# File management
import os
from google.colab import drive

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# mount drive
drive.mount('/content/drive')

# set working directory
os.chdir('/content/drive/MyDrive/WIP/TM&S/progetto TM&S')

Mounted at /content/drive


## 1. Data Import

In [ ]:
# WikiHow whole article dataset
wikiall = pd.read_csv('wikihowAll.csv')
wikiall = wikiall.astype(str)
wikiall

,headline,title,text
0,"\nKeep related supplies in the same area.,\nMa...",How to Be an Organized Artist1,"If you're a photographer, keep all the necess..."
1,\nCreate a sketch in the NeoPopRealist manner ...,How to Create a Neopoprealist Art Work,See the image for how this drawing develops s...
2,"\nGet a bachelor’s degree.,\nEnroll in a studi...",How to Be a Visual Effects Artist1,It is possible to become a VFX artist without...
3,\nStart with some experience or interest in ar...,How to Become an Art Investor,The best art investors do their research on t...
4,"\nKeep your reference materials, sketches, art...",How to Be an Organized Artist2,"As you start planning for a project or work, ..."
...,...,...,...
215360,\nConsider changing the spelling of your name....,How to Pick a Stage Name3,"If you have a name that you like, you might f..."
215361,"\nTry out your name.,\nDon’t legally change yo...",How to Pick a Stage Name4,Your name might sound great to you when you s...
215362,"\nUnderstand the process of relief printing.,\...",How to Identify Prints1,Relief printing is the oldest and most tradit...
215363,\nUnderstand the process of intaglio printing....,How to Identify Prints2,"Intaglio is Italian for ""incis­ing,"" and corr..."


In [ ]:
# WikiHow article divided by paragraph dataset
wikisep = pd.read_csv('wikihowSep.csv',dtype="string")
wikisep

,overview,headline,text,sectionLabel,title
0,So you're a new or aspiring artist and your c...,Sell yourself first.,"Before doing anything else, stop and sum up y...",Steps,How to Sell Fine Art Online
1,"If you want to be well-read, then, in the wor...",Read the classics before 1600.,Reading the classics is the very first thing ...,Reading the Classics,How to Be Well Read
2,So you're a new or aspiring artist and your c...,Join online artist communities.,Depending on what scale you intend to sell yo...,Steps,How to Sell Fine Art Online
3,So you're a new or aspiring artist and your c...,Make yourself public.,Get yourself out there as best as you can by ...,Steps,How to Sell Fine Art Online
4,So you're a new or aspiring artist and your c...,Blog about your artwork.,"Given the hundreds of free blogging websites,...",Steps,How to Sell Fine Art Online
...,...,...,...,...,...
1585690,"Before digital and photographic technology, i...",Magnify the image.,Unlike some of the other varieties of pre-pho...,Identifying Planographic Lithographs,How to Identify Prints3
1585691,"Before digital and photographic technology, i...",Look for the absence of plate marks.,"If you find plate marks, you're always dealin...",Identifying Planographic Lithographs,How to Identify Prints3
1585692,"Before digital and photographic technology, i...",Look for the flatness of the ink.,"Upon close examination, you should notice tha...",Identifying Planographic Lithographs,How to Identify Prints3
1585693,"Before digital and photographic technology, i...","Look for the illusion of shade, created by mu...",Since the planographic surface holds and repe...,Identifying Planographic Lithographs,How to Identify Prints3


## 2. Text Pre-processing

### WikiHow All

#### Feature selection

In [ ]:
wikiall.drop('title', axis=1, inplace=True)
wikiall.head()

,headline,text
0,"\nKeep related supplies in the same area.,\nMa...","If you're a photographer, keep all the necess..."
1,\nCreate a sketch in the NeoPopRealist manner ...,See the image for how this drawing develops s...
2,"\nGet a bachelor’s degree.,\nEnroll in a studi...",It is possible to become a VFX artist without...
3,\nStart with some experience or interest in ar...,The best art investors do their research on t...
4,"\nKeep your reference materials, sketches, art...","As you start planning for a project or work, ..."


#### Null values

In [ ]:
null_values = wikiall.isnull().sum()

print("Null values in each column:")
print(null_values)

Null values in each column:
headline    0
text        0
dtype: int64


#### Case folding

In [ ]:
for row in range(len(wikiall)):
    article = wikiall.loc[row, 'text']
    abstract = wikiall.loc[row,'headline']
    article = article.lower()
    abstract = abstract.lower()

    wikiall.at[row, 'text'] = article
    wikiall.at[row, 'headline'] = abstract

wikiall.head()

,headline,text
0,"\nkeep related supplies in the same area.,\nma...","if you're a photographer, keep all the necess..."
1,\ncreate a sketch in the neopoprealist manner ...,see the image for how this drawing develops s...
2,"\nget a bachelor’s degree.,\nenroll in a studi...",it is possible to become a vfx artist without...
3,\nstart with some experience or interest in ar...,the best art investors do their research on t...
4,"\nkeep your reference materials, sketches, art...","as you start planning for a project or work, ..."


#### Duplicates

In [ ]:
wikiall.drop_duplicates(inplace=True)
wikiall.reset_index(drop=True, inplace=True)
wikiall

,headline,text
0,"\nkeep related supplies in the same area.,\nma...","if you're a photographer, keep all the necess..."
1,\ncreate a sketch in the neopoprealist manner ...,see the image for how this drawing develops s...
2,"\nget a bachelor’s degree.,\nenroll in a studi...",it is possible to become a vfx artist without...
3,\nstart with some experience or interest in ar...,the best art investors do their research on t...
4,"\nkeep your reference materials, sketches, art...","as you start planning for a project or work, ..."
...,...,...
214453,\nconsider changing the spelling of your name....,"if you have a name that you like, you might f..."
214454,"\ntry out your name.,\ndon’t legally change yo...",your name might sound great to you when you s...
214455,"\nunderstand the process of relief printing.,\...",relief printing is the oldest and most tradit...
214456,\nunderstand the process of intaglio printing....,"intaglio is italian for ""incis­ing,"" and corr..."


#### Remove Links

In [ ]:
def remove_links(text):
  remove_https = re.sub(r'https?://\S+|www\.\S+', ' ', text)
  remove_com = re.sub(r"\.com", " ", remove_https)
  return remove_com

for row in range(len(wikiall)):
    article = wikiall.loc[row, 'text']
    abstract = wikiall.loc[row,'headline']
    article = remove_links(article)
    abstract = remove_links(abstract)

    wikiall.at[row, 'text'] = article
    wikiall.at[row, 'headline'] = abstract

#### Remove HTML tags

In [ ]:
from bs4 import BeautifulSoup

for row in range(len(wikiall)):
    article = wikiall.loc[row, 'text']
    abstract = wikiall.loc[row, 'headline']

    article_soup = BeautifulSoup(article, 'html.parser')
    abstract_soup = BeautifulSoup(abstract, 'html.parser')

    article = article_soup.get_text(separator=' ')
    abstract = abstract_soup.get_text(separator=' ')

    wikiall.at[row, 'text'] = article
    wikiall.at[row, 'headline'] = abstract

C:\Users\baghr\AppData\Local\Temp\ipykernel_5656\3046382222.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  abstract_soup = BeautifulSoup(abstract, 'html.parser')
C:\Users\baghr\AppData\Local\Temp\ipykernel_5656\3046382222.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  article_soup = BeautifulSoup(article, 'html.parser')
C:\Users\baghr\anaconda3\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument

#### Contractions

In [ ]:
for row in range(len(wikiall)):
    article = wikiall.loc[row, 'text']
    abstract = wikiall.loc[row,'headline']

    article = contractions.fix(article)
    abstract = contractions.fix(abstract)

    wikiall.at[row, 'text'] = article
    wikiall.at[row, 'headline'] = abstract

#### Remove numbers

In [ ]:
for row in range(len(wikiall)):
    article = wikiall.loc[row, 'text']
    abstract = wikiall.loc[row, 'headline']

    article_without_numbers = re.sub(r'\d+', '', article)
    abstract_without_numbers = re.sub(r'\d+', '', abstract)

    wikiall.at[row, 'text'] = article_without_numbers
    wikiall.at[row, 'headline'] = abstract_without_numbers

#### Remove punctuation and other symbols

In [ ]:
def remove_punctuation(text):
    punctuation_free = ''.join([i if i == '.' or i not in string.punctuation else ' ' for i in text])
    return punctuation_free

for row in range(len(wikiall)):
    article = wikiall.loc[row, 'text']
    abstract = wikiall.loc[row,'headline']
    article = remove_punctuation(article)
    abstract = remove_punctuation(abstract)
    article = article.replace('\n', ' ')
    abstract = abstract.replace('\n', ' ')

    wikiall.at[row, 'text'] = article
    wikiall.at[row, 'headline'] = abstract

#### Whitespaces

In [ ]:
articles_with_whitespaces_removed = 0
articles_without_whitespaces_removed = 0
abstracts_with_whitespaces_removed = 0
abstracts_without_whitespaces_removed = 0

for row in range(len(wikiall)):
    article = wikiall.loc[row, 'text']
    abstract = wikiall.loc[row,'headline']
    modified_article = re.sub(r'\s\s+', ' ', article).strip()
    modified_abstract = re.sub(r'\s\s+', ' ', abstract).strip()

    # Check if whitespaces were removed in articles
    if len(article) != len(modified_article):
        articles_with_whitespaces_removed += 1
    else:
        articles_without_whitespaces_removed += 1

    # Check if whitespaces were removed in abstracts
    if len(abstract) != len(modified_abstract):
        abstracts_with_whitespaces_removed += 1
    else:
        abstracts_without_whitespaces_removed += 1

    # Updating the dataframe
    wikiall.at[row, 'text'] = modified_article
    wikiall.at[row, 'headline'] = modified_abstract

print("Number of articles with whitespaces removed:", articles_with_whitespaces_removed)
print("Number of articles without extra whitespaces:", articles_without_whitespaces_removed)
print("Number of abstracts with whitespaces removed:", abstracts_with_whitespaces_removed)
print("Number of abstracts without extra whitespaces:", abstracts_without_whitespaces_removed)

Number of articles with whitespaces removed: 214205
Number of articles without extra whitespaces: 253
Number of abstracts with whitespaces removed: 214415
Number of abstracts without extra whitespaces: 43


#### Stop words

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baghr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.tokenize import RegexpTokenizer

stop_words = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    tokens = nltk.regexp_tokenize(text, r'\w+|\.')
    filtered_tokens = [token for token in tokens if token not in stop_words]
    text = ' '.join(filtered_tokens)
    return text

for row in range(len(wikiall)):
    article = wikiall.loc[row, 'text']
    abstract = wikiall.loc[row,'headline']

    article = remove_stopwords(article)
    abstract = remove_stopwords(abstract)

    wikiall.at[row, 'text'] = article
    wikiall.at[row, 'headline'] = abstract

#### Tokenization

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\baghr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.tokenize import sent_tokenize

# Create new columns for tokenized sentences
wikiall['headline_sentences'] = None
wikiall['text_sentences'] = None

for row in range(len(wikiall)):
    article = wikiall.loc[row, 'text']
    abstract = wikiall.loc[row, 'headline']

    article_sentences = sent_tokenize(article)
    abstract_sentences = sent_tokenize(abstract)

    wikiall.at[row, 'headline_sentences'] = abstract_sentences
    wikiall.at[row, 'text_sentences'] = article_sentences

#### Stemmatization

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\baghr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
stemmer = PorterStemmer()

#defining a function for stemming
def stemming(text):
    tokens = nltk.regexp_tokenize(text, r'\w+')
    stem_text = [stemmer.stem(token) for token in tokens]
    stem_text = ' '.join(stem_text)
    return stem_text

for row in range(len(wikiall)):
    article = wikiall.loc[row, 'text']
    abstract = wikiall.loc[row, 'headline']

    stemmed_article = stemming(article)
    stemmed_abstract = stemming(abstract)

    wikiall.at[row, 'stemmed_article'] = stemmed_article
    wikiall.at[row, 'stemmed_abstract'] = stemmed_abstract

C:\Users\baghr\AppData\Local\Temp\ipykernel_5656\328842431.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'photograph keep necessari len cord batteri quadrant home studio . paint kept brush cleaner canva print suppli ink etc . make broader group area suppli make find easier limit search much smaller area . idea includ essenti suppli area thing use everi day . inspir refer area . dedic work area . infrequ secondari suppli area tuck way . mean clean entir studio mean keep area immedi around desk easel potteri wheel etc . clean night . discard trash unnecessari materi wipe dirti surfac . endeavor leav workspac way sit next day start work immedi without work tidi . even rest studio bit disorgan organ workspac help get busi everi time want make art . visual peopl lot artist clutter come desir keep track suppli visual instead tuck sight . use jar old glass vase cheap clear plastic drawer keep thing sight without lea

#### Lemmatization

In [ ]:
lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatize_text(text):
    tokens = nltk.regexp_tokenize(text, r'\w+')
    lemm_text = [lemmatizer.lemmatize(token) for token in tokens]
    lemmatized_text = ' '.join(lemm_text)
    return lemmatized_text

for row in range(len(wikiall)):
    article = wikiall.loc[row, 'text']
    abstract = wikiall.loc[row, 'headline']

    lemmatized_article = lemmatize_text(article)
    lemmatized_abstract = lemmatize_text(abstract)

    wikiall.at[row, 'lemmatized_article'] = lemmatized_article
    wikiall.at[row, 'lemmatized_abstract'] = lemmatized_abstract

In [ ]:
# wikiall.to_csv('wikiall_processed.csv')

### WikiHow Sep

#### Feature selection and renaming

In [ ]:
wikisep.drop(columns=['overview','sectionLabel'], inplace=True)
wikisep.rename(columns={'headline':'y'}, inplace=True)
wikisep = wikisep.reindex(columns=['title','text','y'])
wikisep.head()

,title,text,y
0,How to Sell Fine Art Online,"Before doing anything else, stop and sum up y...",Sell yourself first.
1,How to Be Well Read,Reading the classics is the very first thing ...,Read the classics before 1600.
2,How to Sell Fine Art Online,Depending on what scale you intend to sell yo...,Join online artist communities.
3,How to Sell Fine Art Online,Get yourself out there as best as you can by ...,Make yourself public.
4,How to Sell Fine Art Online,"Given the hundreds of free blogging websites,...",Blog about your artwork.


#### Missing values

In [ ]:
# % of missing values by column
wikisep.isna().sum()*100/len(wikisep)

title     0.000063
text     12.512179
y         0.000000
dtype: float64

In [ ]:
wikisep.dropna(axis=0, inplace=True)
wikisep.reset_index(drop=True, inplace=True)
wikisep.head()

,title,text,y
0,How to Sell Fine Art Online,"Before doing anything else, stop and sum up y...",Sell yourself first.
1,How to Be Well Read,Reading the classics is the very first thing ...,Read the classics before 1600.
2,How to Sell Fine Art Online,Depending on what scale you intend to sell yo...,Join online artist communities.
3,How to Sell Fine Art Online,Get yourself out there as best as you can by ...,Make yourself public.
4,How to Sell Fine Art Online,"Given the hundreds of free blogging websites,...",Blog about your artwork.


In [ ]:
# % of missing values by column
wikisep.isna().sum()*100/len(wikisep)

title    0.0
text     0.0
y        0.0
dtype: float64

#### Duplicates

In [ ]:
n_duplicates = len(wikisep) - len(wikisep.drop_duplicates())
wikisep.drop_duplicates(inplace=True)
wikisep.reset_index(drop=True, inplace=True)
print(str(n_duplicates)+' records were dropped')
wikisep

657 records were dropped


,title,text,y
0,How to Sell Fine Art Online,"Before doing anything else, stop and sum up y...",Sell yourself first.
1,How to Be Well Read,Reading the classics is the very first thing ...,Read the classics before 1600.
2,How to Sell Fine Art Online,Depending on what scale you intend to sell yo...,Join online artist communities.
3,How to Sell Fine Art Online,Get yourself out there as best as you can by ...,Make yourself public.
4,How to Sell Fine Art Online,"Given the hundreds of free blogging websites,...",Blog about your artwork.
...,...,...,...
1386627,How to Identify Prints3,Unlike some of the other varieties of pre-pho...,Magnify the image.
1386628,How to Identify Prints3,"If you find plate marks, you're always dealin...",Look for the absence of plate marks.
1386629,How to Identify Prints3,"Upon close examination, you should notice tha...",Look for the flatness of the ink.
1386630,How to Identify Prints3,Since the planographic surface holds and repe...,"Look for the illusion of shade, created by mu..."


#### Basic text cleaning

In [ ]:
## Helper functions

# remove (spece + punctuation) and (punctuation + space) sequences
def Remove_extra_punct(s):
  s = s.lstrip((punctuation) + ' ')
  s = s.rstrip((punctuation) + ' ')
  return s.strip()

# clean text from unwanted character sequences in preparation for the parser
# corrects for inapropriate spacing, repeated punctuation, and remove web links
# preserves functional punctuation within sentences and delimits sentences with '.'
# does not remuve number nor words or capitalization
def Clean_document(doc):
  doc = re.sub(r'\n|\r', ' ', doc)
  doc = re.sub(r'(\W)(?=\1)', '', doc)
  doc = re.sub(r' +', ' ', doc)
  doc = re.sub(r'^https?:\/\/.*[\r\n]*', '', doc, flags=re.MULTILINE)
  doc = re.sub(r'\.com', '', doc)
  doc = re.sub(r'(?<=[.,])(?=[^\s])', r' ', doc)
  doc = re.sub(r'\[[^\]]*\]', ' ', doc)
  doc = doc.strip()
  sentences = nltk.sent_tokenize(doc)
  clean_sentences = [Remove_extra_punct(sent) for sent in sentences]
  clean_sentences = np.delete(clean_sentences, np.where(np.array(clean_sentences) == '')[0])
  doc = '.'.join(list(clean_sentences))
  doc = re.sub(r'(?<=[.,])(?=[^\s])', r' ', doc)
  return  doc+'.'

In [ ]:
# exemple: before
wikisep.text.iloc[0]

" Before doing anything else, stop and sum up yourself as an artist. Now, think about how to translate that to an online profile. Be it the few words, Twitter allows you or an entire page of indulgence that your own website would allow you. Bring out the most salient features of your creativity, your experience, your passion, and your reasons for painting. Make it clear to readers why you are an artist who loves art, produces high quality art, and is a true champion of art. If you're not great with words, find a friend who can help you with this really important aspect of selling online – the establishment of your credibility and reliability.;\n"

In [ ]:
# exemple: after
Clean_document(wikisep.text.iloc[0])

"Before doing anything else, stop and sum up yourself as an artist. Now, think about how to translate that to an online profile. Be it the few words, Twitter allows you or an entire page of indulgence that your own website would allow you. Bring out the most salient features of your creativity, your experience, your passion, and your reasons for painting. Make it clear to readers why you are an artist who loves art, produces high quality art, and is a true champion of art. If you're not great with words, find a friend who can help you with this really important aspect of selling online – the establishment of your credibility and reliability."

In [ ]:
# # apply Clean_document to text and y for all record and store results in wikisep as text_clean and y_clean
# wikisep['text_clean'] = [Clean_document(x) for x in tqdm(wikisep['text'])]
# wikisep['y_clean'] = [Clean_document(x) for x in tqdm(wikisep['y'])]
# wikisep.head()

,title,text,y,text_clean,y_clean
0,How to Sell Fine Art Online,"Before doing anything else, stop and sum up y...",Sell yourself first.,"Before doing anything else, stop and sum up yo...",Sell yourself first.
1,How to Be Well Read,Reading the classics is the very first thing ...,Read the classics before 1600.,Reading the classics is the very first thing y...,Read the classics before 1600.
2,How to Sell Fine Art Online,Depending on what scale you intend to sell yo...,Join online artist communities.,Depending on what scale you intend to sell you...,Join online artist communities.
3,How to Sell Fine Art Online,Get yourself out there as best as you can by ...,Make yourself public.,Get yourself out there as best as you can by a...,Make yourself public.
4,How to Sell Fine Art Online,"Given the hundreds of free blogging websites,...",Blog about your artwork.,"Given the hundreds of free blogging websites, ...",Blog about your artwork.


In [ ]:
# # clean wikisep export
# wikisep.to_csv('wikisep_processed.csv')